In [21]:
# Importing Libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

In [22]:
# Setting up the variables
game_ids_87 = []
game_ids_86 = []
team_score = []
team_points = []
teams = []
stats = []
wins = []

In [23]:
### SCRAPING SZN 87 DATA ###

# Setting up Chrome options
options = Options()
options.add_argument("--headless")  # Run in headless mode

# Initializing WebDriver
driver = webdriver.Chrome(options=options)

# Navigating to the target URL
url = 'https://uaapvolleyball.livestats.ph/tournaments/uaap-volleyball-season-87-women-s'
driver.get(url)

# Waiting for the page to load content
time.sleep(5)

# Extracting Game IDs
game_cards_87 = driver.find_elements(By.CSS_SELECTOR, "a[href*='/games/']")

for box in game_cards_87:
    href = box.get_attribute("href")
    match = re.search(r"/games/(\d+)/stats", href)
    if match:
        game_id = match.group(1)
        game_ids_87.append(game_id)

# Extracting Stats per Game ID
base_url = "https://uaapvolleyball.livestats.ph/tournaments/uaap-volleyball-season-87-women-s"

for game_id in game_ids_87:
    full_url = f"{base_url}?game_id={game_id}"
    driver.get(full_url)
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    score_raw = soup.find_all('div', class_='team_score')
    points_raw = soup.find_all('td', class_='period-score')
    teams_raw = soup.find_all('div', class_='team_name')
    stats_raw = soup.find_all('tr', class_='team-totals')

    sets = []
    for d in score_raw:
        sets.append(d.text.strip())
        team_score.append(d.text.strip())

    if sets[0] > sets[1]:
        wins.append(1)
        wins.append(0)
    else:
        wins.append(0)
        wins.append(1)

    raw_points = []
    for d in points_raw:
        raw_points.append(d.get_text(strip=True))
        
    mid_index = len(raw_points) // 2
    team_points.append(sum(list(map(int, raw_points[:mid_index]))))
    team_points.append(sum(list(map(int, raw_points[mid_index:]))))

    for d in teams_raw[:2]:
        teams.append(d.text.strip())

    for d in stats_raw:
        stats.append(d.text.strip())

# Closing the browser
driver.quit()

In [24]:
### SCRAPING SZN 86 DATA ###

# Setting up Chrome options
options = Options()
options.add_argument("--headless")  # Run in headless mode

# Initializing WebDriver
driver = webdriver.Chrome(options=options)

# Navigating to the target URL
url = "https://uaapvolleyball.livestats.ph/tournaments/uaap-volleyball-season-86-women-s"
driver.get(url)

# Waiting for the page to load content
time.sleep(5)  # Adjust sleep time as needed

# Extracting Game IDs
game_cards_86 = driver.find_elements(By.CSS_SELECTOR, "a[href*='/games/']")

for box in game_cards_86:
    href = box.get_attribute("href")
    match = re.search(r"/games/(\d+)/stats", href)
    if match:
        game_id = match.group(1)
        game_ids_86.append(game_id)

#Extracting Stats per Game ID
base_url = "https://uaapvolleyball.livestats.ph/tournaments/uaap-volleyball-season-86-women-s"

for game_id in game_ids_86:
    full_url = f"{base_url}?game_id={game_id}"
    driver.get(full_url)
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    score_raw = soup.find_all('div', class_='team_score')
    points_raw = soup.find_all('td', class_='period-score')
    teams_raw = soup.find_all('div', class_='team_name')
    stats_raw = soup.find_all('tr', class_='team-totals')

    sets = []
    for d in score_raw:
        sets.append(d.text.strip())
        team_score.append(d.text.strip())

    if sets[0] > sets[1]:
        wins.append(1)
        wins.append(0)
    else:
        wins.append(0)
        wins.append(1)

    raw_points = []
    for d in points_raw:
        raw_points.append(d.get_text(strip=True))
        
    mid_index = len(raw_points) // 2
    team_points.append(sum(list(map(int, raw_points[:mid_index]))))
    team_points.append(sum(list(map(int, raw_points[mid_index:]))))

    for d in teams_raw[:2]:
        teams.append(d.text.strip())

    for d in stats_raw:
        stats.append(d.text.strip())

# Closing the browser
driver.quit()

In [ ]:
# Splitting the stats list by \n
stats = [item.split('\n') for item in stats]

# Setting up the seasons variable
seasons = ['87'] * (len(game_ids_87) * 2)
seasons.extend(['86'] * (len(game_ids_86) * 2))

In [ ]:
# Converting into DataFrame
df = pd.DataFrame(stats)
df[0] = teams
df.columns = ['school', 'attack', 'block', 'serve', 'dig', 'receive', 'set']

df['season'] = seasons

ids = []
for i in game_ids_87:
    ids.append(i)
    ids.append(i)
for i in game_ids_86:
    ids.append(i)
    ids.append(i)
df['game_id'] = ids

df['win'] = wins
df['sets_won'] = team_score
df['total_points'] = team_points

volley_stats = ['attack', 'block', 'serve', 'dig', 'receive', 'set']
for i in volley_stats:
    col_1 = i + '_success'
    col_2 = i + '_attempted'
    df[[col_1, col_2]] = df[i].str.split('/', expand=True)
df = df.drop(columns=volley_stats)

,school,season,game_id,win,sets_won,total_points,attack_success,attack_attempted,block_success,block_attempted,serve_success,serve_attempted,dig_success,dig_attempted,receive_success,receive_attempted,set_success,set_attempted
0,DE LA SALLE UNIVERSITY,87,126,0,0,56,26,147,8,53,3,57,35,120,32,68,12,121
1,NATIONAL UNIVERSITY,87,126,1,3,75,48,143,7,47,2,74,53,122,23,53,17,125
2,NATIONAL UNIVERSITY,87,124,1,3,88,62,172,4,66,4,87,50,149,43,73,25,147
3,DE LA SALLE UNIVERSITY,87,124,0,1,80,47,154,7,66,4,81,40,140,38,75,19,138
4,DE LA SALLE UNIVERSITY,87,121,1,3,86,46,173,11,71,7,85,53,144,34,82,17,154


In [ ]:
# df.to_csv(r'folder/destination/uaap-ph-volleyball', index = False)